In [8]:
import sys
sys.path.append('../')
from simCRN.multivariate_reg import read_eq_data_file
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import tensorflow_docs as tfdocs
import tensorflow_docs.plots
import tensorflow_docs.modeling
import matplotlib.pyplot as plt
import numpy as np
import math
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_absolute_error as mae
from sklearn.metrics import mean_squared_error as mse
from sklearn.metrics import r2_score
from hyperopt import hp
from hyperopt import fmin, tpe, space_eval, Trials
from sklearn.model_selection import cross_val_score

# Baseline model: Random Forest

Reading in the data

In [2]:
Ci_all_array, Am_array, Cmin, Cmax, Ai = read_eq_data_file('4-4-2-semi-asym-AB-AC.txt')

In [20]:
Am_array.shape

(2000, 4)

Prepare the data

In [3]:
# Splitting into train and test
X_train, X_test, y_train, y_test = train_test_split(Am_array, Ci_all_array, test_size=0.2, random_state=0)

# Z normalizing
X_scaler = StandardScaler().fit(X_train)
X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

Random Forest Model

In [4]:
random_forest_model = RandomForestRegressor(criterion="squared_error", random_state=0)
random_forest_model.fit(X_train_scaled, y_train)
y_hat_train = random_forest_model.predict(X_train_scaled)
y_hat_test = random_forest_model.predict(X_test_scaled)

In [5]:
train_mae = mae(y_train, y_hat_train, multioutput='raw_values')
test_mae = mae(y_test, y_hat_test, multioutput='raw_values')

print(f'The MAE on the training data for C₁ is {train_mae[0]:.3}') # 3 significant figures
print(f'The MAE on the training data for C₂ is {train_mae[1]:.3}')
print(f'The MAE on the test data for C₁ is {test_mae[0]:.3}')
print(f'The MAE on the test data for C₂ is {test_mae[1]:.3}')

print() # new line

# Contextualizing with the mean of C₁ and C₂
means = np.mean(Ci_all_array, axis=0)
print(f'The average value of C₁ is {means[0]:.3}')
print(f'The average value of C₂ is {means[1]:.3}')

print() # new line

print(f'For the test data, MAE/mean for C₁ is {test_mae[0]/means[0]:.3}')
print(f'For the test data, MAE/mean for C₂ is {test_mae[1]/means[1]:.3}')

The MAE on the training data for C₁ is 1.06e-08
The MAE on the training data for C₂ is 1.03e-08
The MAE on the test data for C₁ is 2.35e-08
The MAE on the test data for C₂ is 2.39e-08

The average value of C₁ is 1.51e-06
The average value of C₂ is 1.48e-06

For the test data, MAE/mean for C₁ is 0.0155
For the test data, MAE/mean for C₂ is 0.0162


In [6]:
train_mse = mse(y_train, y_hat_train, multioutput='raw_values')
test_mse = mse(y_test, y_hat_test, multioutput='raw_values')
train_rmse = np.sqrt(train_mse)
test_rmse = np.sqrt(test_mse)

print(f'The RMSE on the training data for C₁ is {train_rmse[0]:.3}') # 3 significant figures
print(f'The RMSE on the training data for C₂ is {train_rmse[1]:.3}')
print(f'The RMSE on the test data for C₁ is {test_rmse[0]:.3}')
print(f'The RMSE on the test data for C₂ is {test_rmse[1]:.3}')

print() # new line

# Contextualizing with the mean of C₁ and C₂
print(f'The average value of C₁ is {means[0]:.3}')
print(f'The average value of C₂ is {means[1]:.3}')

print() # new line

print(f'For the test data, RMSE/mean for C₁ is {test_rmse[0]/means[0]:.3}')
print(f'For the test data, RMSE/mean for C₂ is {test_rmse[1]/means[1]:.3}')

The RMSE on the training data for C₁ is 1.38e-08
The RMSE on the training data for C₂ is 1.35e-08
The RMSE on the test data for C₁ is 3.05e-08
The RMSE on the test data for C₂ is 3.12e-08

The average value of C₁ is 1.51e-06
The average value of C₂ is 1.48e-06

For the test data, RMSE/mean for C₁ is 0.0201
For the test data, RMSE/mean for C₂ is 0.0211


In [9]:
train_r2 = r2_score(y_train, y_hat_train, multioutput='raw_values')
test_r2 = r2_score(y_test, y_hat_test, multioutput='raw_values')

print(f'The R² on the training data for C₁ is {train_r2[0]:.3}') # 3 significant figures
print(f'The R² on the training data for C₂ is {train_r2[1]:.3}')
print(f'The R² on the test data for C₁ is {test_r2[0]:.3}')
print(f'The R² on the test data for C₂ is {test_r2[1]:.3}')

The R² on the training data for C₁ is 1.0
The R² on the training data for C₂ is 1.0
The R² on the test data for C₁ is 0.999
The R² on the test data for C₂ is 0.999


# Hyperparameter optimization

In [10]:
n_estimators_list = [20, 40, 80, 100, 160, 320, 640, 1280]
min_samples_split_list = [2, 8, 10, 12, 24]
max_depth_list = [2, 4, 8, 10, None]
max_depth_list = [2, 4, 8, 10, None]
parameter_space = {'n_estimators': hp.choice('n_estimators', n_estimators_list), \
                   'min_samples_split': hp.choice('min_samples_split', min_samples_split_list), \
                   'max_depth': hp.choice('max_depth', max_depth_list)}

In [11]:
print(f"Parameter Space Size: {len(n_estimators_list)*len(min_samples_split_list)*len(max_depth_list)}")

Parameter Space Size: 200


In [12]:
from hyperopt import hp
from hyperopt import fmin, tpe, space_eval, Trials
from sklearn.model_selection import cross_val_score

In [13]:
def model_eval(args):

    '''Take suggested arguments and perform model evaluation'''
    
    model = RandomForestRegressor(criterion="squared_error", n_estimators=args['n_estimators'], \
                                  min_samples_split=args['min_samples_split'], max_depth=args['max_depth'])
    
    scores = cross_val_score(model, X_train_scaled, y=y_train, scoring='neg_mean_squared_error')

    cv_score = np.mean(scores)

    # return the negative of the CV score to ensure we maximize the negative MSE by minimizing the loss
    return -cv_score

In [14]:
print("Start trials") 

trials = Trials()
best = fmin(model_eval, parameter_space, algo=tpe.suggest, max_evals=200, trials=trials)

Start trials
100%|██████████| 200/200 [36:59<00:00, 11.10s/trial, best loss: 1.352862235430332e-15]


In [19]:
print("Best parameter set: {}".format(best))
print("Best loss from CV: {:.3}".format(trials.best_trial['result']['loss']))
print("Best RMSE loss from CV: {:.3}".format(np.sqrt(trials.best_trial['result']['loss'])))

Best parameter set: {'max_depth': 4, 'min_samples_split': 0, 'n_estimators': 5}
Best loss from CV: 1.35e-15
Best RMSE loss from CV: 3.68e-08


In [33]:
print("Best parameters")
print(f"n_estimators = {n_estimators_list[best['n_estimators']]}")
print(f"max_depth = {max_depth_list[best['max_depth']]}")
print(f"min_samples_split = {min_samples_split_list[best['min_samples_split']]}")

Best parameters
n_estimators = 320
max_depth = None
min_samples_split = 2


In [17]:
optimized_random_forest_model = RandomForestRegressor(criterion="squared_error", \
                                                      n_estimators=n_estimators_list[best['n_estimators']], 
                                                      max_depth=max_depth_list[best['max_depth']], \
                                                      min_samples_split=min_samples_split_list[best['min_samples_split']])
optimized_random_forest_model.fit(X_train_scaled, y_train)
optimized_y_hat_train = optimized_random_forest_model.predict(X_train_scaled)
optimized_y_hat_test = optimized_random_forest_model.predict(X_test_scaled)

In [18]:
optimized_train_mse = mse(y_train, optimized_y_hat_train, multioutput='raw_values')
optimized_test_mse = mse(y_test, optimized_y_hat_test, multioutput='raw_values')
optimized_train_rmse = np.sqrt(optimized_train_mse)
optimized_test_rmse = np.sqrt(optimized_test_mse)

print(f'The RMSE on the training data for C₁ is {optimized_train_rmse[0]:.3}') # 3 significant figures
print(f'The RMSE on the training data for C₂ is {optimized_train_rmse[1]:.3}')
print(f'The RMSE on the test data for C₁ is {optimized_test_rmse[0]:.3}')
print(f'The RMSE on the test data for C₂ is {optimized_test_rmse[1]:.3}')

print() # new line

# Contextualizing with the mean of C₁ and C₂
print(f'The average value of C₁ is {means[0]:.3}')
print(f'The average value of C₂ is {means[1]:.3}')

print() # new line

print(f'For the test data, RMSE/mean for C₁ is {optimized_test_rmse[0]/means[0]:.3}')
print(f'For the test data, RMSE/mean for C₂ is {optimized_test_rmse[1]/means[1]:.3}')

The RMSE on the training data for C₁ is 1.28e-08
The RMSE on the training data for C₂ is 1.24e-08
The RMSE on the test data for C₁ is 2.94e-08
The RMSE on the test data for C₂ is 2.99e-08

The average value of C₁ is 1.51e-06
The average value of C₂ is 1.48e-06

For the test data, RMSE/mean for C₁ is 0.0194
For the test data, RMSE/mean for C₂ is 0.0202


# Reducing Dataset Size

In [21]:
def prep_data(Am_array, Ci_all_array, total_size, test_size, random_state):
    # Reducing the dataset size
    X_train_all, X_test_all, y_train_all, y_test_all = \
    train_test_split(Am_array, Ci_all_array, test_size=1-total_size, random_state=random_state)
    
    # Splitting into train and test
    X_train, X_test, y_train, y_test = \
    train_test_split(X_train_all, y_train_all, test_size=test_size, random_state=random_state)
    
    # Z normalizing
    X_scaler = StandardScaler().fit(X_train)
    X_train_scaled = X_scaler.transform(X_train)
    X_test_scaled = X_scaler.transform(X_test)
    return(X_train_scaled, X_test_scaled, X_test_all, y_train, y_test, y_test_all, X_scaler)

In [34]:
# Reduced data set sizes
data_set_sizes = 0.5 ** np.arange(1,6)
print(data_set_sizes*2000)
# Lists to store data and results in
trials_list = []
best_params_list = []
X_train_scaled_list = []
X_test_scaled_list = []
X_test_all_list = []
y_train_list = []
y_test_list = []
y_test_all_list = []
X_scaler_list = []

[1000.   500.   250.   125.    62.5]


In [35]:
# Iterate through smaller data set sizes
for data_set_size in data_set_sizes:
    print(f"Data set size: {data_set_size*2000}")
    X_train_scaled, X_test_scaled, X_test_all, y_train, y_test, y_test_all, X_scaler = \
    prep_data(Am_array, Ci_all_array, data_set_size, 0.2, 0)
    # Add data sets to lists
    X_train_scaled_list.append(X_train_scaled)
    X_test_scaled_list.append(X_test_scaled)
    X_test_all_list.append(X_test_all)
    y_train_list.append(y_train)
    y_test_list.append(y_test)
    y_test_all_list.append(y_test_all)
    X_scaler_list.append(X_scaler)
    def model_eval_data_set(args):

        '''Take suggested arguments and perform model evaluation'''

        model = RandomForestRegressor(criterion="squared_error", n_estimators=args['n_estimators'], \
                                      min_samples_split=args['min_samples_split'], max_depth=args['max_depth'])

        scores = cross_val_score(model, X_train_scaled, y=y_train, scoring='neg_mean_squared_error')

        cv_score = np.mean(scores)

        # return the negative of the CV score to ensure we maximize the negative MSE by minimizing the loss
        return -cv_score
    # Hyperparameter optimize
    trials = Trials()
    best = fmin(model_eval_data_set, parameter_space, algo=tpe.suggest, max_evals=200, trials=trials)
    trials_list.append(trials)
    best_params_list.append(best)
    print("Best parameter set: {}".format(best))
    print(f"n_estimators = {n_estimators_list[best['n_estimators']]}")
    print(f"max_depth = {max_depth_list[best['max_depth']]}")
    print(f"min_samples_split = {min_samples_split_list[best['min_samples_split']]}")

Data set size: 1000.0
100%|██████████| 200/200 [1:06:57<00:00, 20.09s/trial, best loss: 3.2486420729425907e-15]
Best parameter set: {'max_depth': 4, 'min_samples_split': 0, 'n_estimators': 7}
n_estimators = 1280
max_depth = None
min_samples_split = 2
Data set size: 500.0
100%|██████████| 200/200 [43:33<00:00, 13.07s/trial, best loss: 6.883233265143629e-15]  
Best parameter set: {'max_depth': 4, 'min_samples_split': 0, 'n_estimators': 3}
n_estimators = 100
max_depth = None
min_samples_split = 2
Data set size: 250.0
100%|██████████| 200/200 [22:18<00:00,  6.69s/trial, best loss: 1.804728444186624e-14] 
Best parameter set: {'max_depth': 3, 'min_samples_split': 0, 'n_estimators': 5}
n_estimators = 320
max_depth = 10
min_samples_split = 2
Data set size: 125.0
100%|██████████| 200/200 [15:55<00:00,  4.78s/trial, best loss: 3.125876063900783e-14] 
Best parameter set: {'max_depth': 2, 'min_samples_split': 0, 'n_estimators': 2}
n_estimators = 80
max_depth = 8
min_samples_split = 2
Data set size